In [ ]:
import requests
from pydantic import BaseModel, Field
import datetime
from typing import List, Dict
from langchain.agents import initialize_agent, Tool

In [ ]:
import os
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

os.environ['OPENAI_API_KEY'] = ''

In [ ]:
!pip uninstall -y langchain pydantic
!pip install langchain==0.0.319 pydantic==1.10.8 openai==0.28.1

In [ ]:
from langchain.agents import tool

In [ ]:
class student_profile_path(BaseModel):
    path: str = Field(..., description="path to the json file containing all student profiles")

class professor_profile_path(BaseModel):
    path: str = Field(..., description="path to professor's profile")

In [ ]:
def load_student_profiles(path: str):
    with open(path, 'r') as f:
        return json.load(f)

def load_professor_profiles(path: str):
    with open(path, 'r') as f:
        return json.load(f)

def compare_profiles(students, professor):
    model = OpenAI()
    prompt_template = PromptTemplate(
        input_variables=['student','professor'],
        template="Compare the student and professor's profile and returns how well of a fit this application is: {student} {professor}."
    )

load_student_profiles_tool = Tool(
    func=load_student_profiles,
    name="Load Student Profiles",
    description="Loads the student profiles using the json file path provided",
    args_schema=student_profile_path
)

load_professor_profiles_tool = Tool(
    func=load_professor_profiles,
    name="Load Professor Profiles",
    description="Loads the professor profiles using the json file path provided",
    args_schema=professor_profile_path
)

compare_profiles_tool = Tool(
    func=compare_profiles,
    name="Compare Profiles",
    description="Compares the student and professor's profile and returns how well of a fit this application is"
)

In [ ]:
tools = [
   load_student_profiles_tool,
   load_professor_profiles_tool,
   compare_profiles_tool
]
llm = OpenAI(temperature=0)
agent = initialize_agent(tools=tools,llm=llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [ ]:
 student_file = "student_profiles.json"
 professor_file = "professor_profile.json"
 result = agent.run(f"""
    Load the student profiles from {student_file}.
    Load the professor profile from {professor_file}.
    Then, compare the profiles and return the most common student profiles for the professor.
    """)
print(result)